# Regression Week 5: Feature Selection and LASSO (Interpretation)

In this notebook, you will use LASSO to select features, building on a pre-implemented solver for LASSO (using Turi Create, though you can use other solvers). You will:
* Run LASSO with different L1 penalties.
* Choose best L1 penalty using a validation set.
* Choose best L1 penalty using a validation set, with additional constraint on the size of subset.

In the second notebook, you will implement your own LASSO solver, using coordinate descent. 

# Fire up Turi Create

In [1]:
import turicreate

# Load in house sales data

Dataset is from house sales in King County, the region where the city of Seattle, WA is located.

In [2]:
sales = turicreate.SFrame('home_data.sframe/')

# Create new features

As in Week 2, we consider features that are some transformations of inputs.

In [3]:
from math import log, sqrt
sales['sqft_living_sqrt'] = sales['sqft_living'].apply(sqrt)
sales['sqft_lot_sqrt'] = sales['sqft_lot'].apply(sqrt)
sales['bedrooms_square'] = sales['bedrooms']*sales['bedrooms']

# In the dataset, 'floors' was defined with type string, 
# so we'll convert them to float, before creating a new feature.
sales['floors'] = sales['floors'].astype(float) 
sales['floors_square'] = sales['floors']*sales['floors']

* Squaring bedrooms will increase the separation between not many bedrooms (e.g. 1) and lots of bedrooms (e.g. 4) since 1^2 = 1 but 4^2 = 16. Consequently this variable will mostly affect houses with many bedrooms.
* On the other hand, taking square root of sqft_living will decrease the separation between big house and small house. The owner may not be exactly twice as happy for getting a house that is twice as big.

# Learn regression weights with L1 penalty

Let us fit a model with all the features available, plus the features we just created above.

In [4]:
all_features = ['bedrooms', 'bedrooms_square',
                'bathrooms',
                'sqft_living', 'sqft_living_sqrt',
                'sqft_lot', 'sqft_lot_sqrt',
                'floors', 'floors_square',
                'waterfront', 'view', 'condition', 'grade',
                'sqft_above',
                'sqft_basement',
                'yr_built', 'yr_renovated']

Applying L1 penalty requires adding an extra parameter (`l1_penalty`) to the linear regression call in Turi Create. (Other tools may have separate implementations of LASSO.)  Note that it's important to set `l2_penalty=0` to ensure we don't introduce an additional L2 penalty.

In [5]:
model_all = turicreate.linear_regression.create(sales, target='price', features=all_features,
                                                validation_set=None, 
                                                l2_penalty=0., l1_penalty=1e10)

Linear regression:

--------------------------------------------------------

Number of examples          : 21613

Number of features          : 17

Number of unpacked features : 17

...

| 5         | 6        | 0.000002  | 1.635510     | 6869667.895833     | 385998.458623                   |

| 10        | 11       | 0.000002  | 1.757402     | 6842123.232651     | 364204.576180                   |

+-----------+----------+-----------+--------------+--------------------+---------------------------------+

Completed (Iteration limit reached).

This model may not be optimal. To improve it, consider increasing `max_iterations`.

Find what features had non-zero weight.

In [7]:
model_all.coefficients.print_rows(18)

+------------------+-------+--------------------+--------+
|       name       | index |       value        | stderr |
+------------------+-------+--------------------+--------+
|   (intercept)    |  None | 274873.05595049576 |  None  |
|     bedrooms     |  None |        0.0         |  None  |
| bedrooms_square  |  None |        0.0         |  None  |
|    bathrooms     |  None | 8468.531086910105  |  None  |
|   sqft_living    |  None | 24.42072098244546  |  None  |
| sqft_living_sqrt |  None | 350.06055338605546 |  None  |
|     sqft_lot     |  None |        0.0         |  None  |
|  sqft_lot_sqrt   |  None |        0.0         |  None  |
|      floors      |  None |        0.0         |  None  |
|  floors_square   |  None |        0.0         |  None  |
|    waterfront    |  None |        0.0         |  None  |
|       view       |  None |        0.0         |  None  |
|    condition     |  None |        0.0         |  None  |
|      grade       |  None | 842.0680348976231  |  None 

Note that a majority of the weights have been set to zero. So by setting an L1 penalty that's large enough, we are performing a subset selection. 

***QUIZ QUESTION***:
According to this list of weights, which of the features have been chosen? 

# Selecting an L1 penalty

To find a good L1 penalty, we will explore multiple values using a validation set. Let us do three way split into train, validation, and test sets:
* Split our sales data into 2 sets: training and test
* Further split our training data into two sets: train, validation

Be *very* careful that you use seed = 1 to ensure you get the same answer!

In [12]:
(training_and_validation, testing) = sales.random_split(.9,seed=1) # initial train/test split
(training, validation) = training_and_validation.random_split(0.5, seed=1) # split training into train and validate

Next, we write a loop that does the following:
* For `l1_penalty` in [10^1, 10^1.5, 10^2, 10^2.5, ..., 10^7] (to get this in Python, type `np.logspace(1, 7, num=13)`.)
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list.
    * Compute the RSS on VALIDATION data (here you will want to use `.predict()`) for that `l1_penalty`
* Report which `l1_penalty` produced the lowest RSS on validation data.

When you call `linear_regression.create()` make sure you set `validation_set = None`.

Note: you can turn off the print out of `linear_regression.create()` with `verbose = False`

In [13]:
import numpy as np

In [34]:
penalty_set=np.logspace(1,7,num=13)
handle=turicreate.SFrame()
array1=np.array([])
array2=np.array([])
print(penalty_set.size)
for l1_penalty in penalty_set:
    model=turicreate.linear_regression.create(training,
                                              target='price',
                                              features=all_features,
                                             l1_penalty=l1_penalty,
                                             l2_penalty=0,
                                             validation_set=None,verbose = False)
    validation_output=model.predict(validation)
    error=validation_output-validation['price']
    rss=np.dot(error,error)
    array1=np.append(array1,[l1_penalty])
    array2=np.append(array2,[rss])
    print(l1_penalty, '-->', rss)
    

13
10.0 --> 625766285142460.6
31.622776601683793 --> 625766285362393.8
100.0 --> 625766286057885.2
316.22776601683796 --> 625766288257224.9
1000.0 --> 625766295212186.0
3162.2776601683795 --> 625766317206080.6
10000.0 --> 625766386760658.2
31622.776601683792 --> 625766606749278.6
100000.0 --> 625767302791635.2
316227.7660168379 --> 625769507643886.0
1000000.0 --> 625776517727024.4
3162277.6601683795 --> 625799062845466.5
10000000.0 --> 625883719085425.1


In [40]:
handle['l1_penalty']=array1
handle['rss']=array2
handle.sort('rss',ascending=True)
handle.print_rows(13)

+--------------------+-------------------+
|     l1_penalty     |        rss        |
+--------------------+-------------------+
|        10.0        | 625766285142460.6 |
| 31.622776601683793 | 625766285362393.8 |
|       100.0        | 625766286057885.2 |
| 316.22776601683796 | 625766288257224.9 |
|       1000.0       | 625766295212186.0 |
| 3162.2776601683795 | 625766317206080.6 |
|      10000.0       | 625766386760658.2 |
| 31622.776601683792 | 625766606749278.6 |
|      100000.0      | 625767302791635.2 |
| 316227.7660168379  | 625769507643886.0 |
|     1000000.0      | 625776517727024.4 |
| 3162277.6601683795 | 625799062845466.5 |
|     10000000.0     | 625883719085425.1 |
+--------------------+-------------------+
[13 rows x 2 columns]



*** QUIZ QUESTION. *** What was the best value for the `l1_penalty`?

In [44]:
model=turicreate.linear_regression.create(training,
                                              target='price',
                                              features=all_features,
                                             l1_penalty=10,
                                             l2_penalty=0,
                                             validation_set=None,verbose = False)
model1=turicreate.linear_regression.create(training,
                                              target='price',
                                              features=all_features,
                                             l1_penalty=0,
                                             l2_penalty=0,
                                             validation_set=None,verbose = False)

***QUIZ QUESTION***
Also, using this value of L1 penalty, how many nonzero weights do you have?

In [45]:
model.coefficients.print_rows(18)
model1.coefficients.print_rows(18)

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |   18993.4272127706   |  None  |
|     bedrooms     |  None |  7936.9676790313015  |  None  |
| bedrooms_square  |  None |  936.9933681932994   |  None  |
|    bathrooms     |  None |  25409.588934120668  |  None  |
|   sqft_living    |  None |  39.11513637970764   |  None  |
| sqft_living_sqrt |  None |  1124.650212807717   |  None  |
|     sqft_lot     |  None | 0.003483618222989654 |  None  |
|  sqft_lot_sqrt   |  None |  148.25839101140826  |  None  |
|      floors      |  None |  21204.335466950117  |  None  |
|  floors_square   |  None |  12915.524336072433  |  None  |
|    waterfront    |  None |   601905.594545272   |  None  |
|       view       |  None |  93312.85731187189   |  None  |
|    condition     |  None |  6609.035712447216   |  None  |
|      grade       |  No

# Limit the number of nonzero weights

What if we absolutely wanted to limit ourselves to, say, 7 features? This may be important if we want to derive "a rule of thumb" --- an interpretable model that has only a few features in them.

In this section, you are going to implement a simple, two phase procedure to achive this goal:
1. Explore a large range of `l1_penalty` values to find a narrow region of `l1_penalty` values where models are likely to have the desired number of non-zero weights.
2. Further explore the narrow region you found to find a good value for `l1_penalty` that achieves the desired sparsity.  Here, we will again use a validation set to choose the best value for `l1_penalty`.

In [46]:
max_nonzeros = 7

## Exploring the larger range of values to find a narrow range with the desired sparsity

Let's define a wide range of possible `l1_penalty_values`:

In [47]:
l1_penalty_values = np.logspace(8, 10, num=20)

Now, implement a loop that search through this space of possible `l1_penalty` values:

* For `l1_penalty` in `np.logspace(8, 10, num=20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Extract the weights of the model and count the number of nonzeros. Save the number of nonzeros to a list.
        * *Hint: `model.coefficients['value']` gives you an SArray with the parameters you learned.  If you call the method `.nnz()` on it, you will find the number of non-zero parameters!* 

In [48]:
nnz=np.array([])
l1=np.array([])
for l1_penalty in l1_penalty_values:
    model=turicreate.linear_regression.create(training,
                                              target='price',
                                              features=all_features,
                                             l1_penalty=l1_penalty,
                                             l2_penalty=0,
                                             validation_set=None,verbose = False)
    nnz=np.append(nnz,[model.coefficients['value'].nnz()])
    l1=np.append(l1,[l1_penalty])
    

In [53]:
handle_1=turicreate.SFrame({'l1':l1,'nnz':nnz})
handle_1.print_rows(20)

+--------------------+------+
|         l1         | nnz  |
+--------------------+------+
|    100000000.0     | 18.0 |
| 127427498.57031322 | 18.0 |
| 162377673.91887242 | 18.0 |
|  206913808.111479  | 18.0 |
| 263665089.87303555 | 17.0 |
| 335981828.6283788  | 17.0 |
| 428133239.8719396  | 17.0 |
| 545559478.1168514  | 17.0 |
| 695192796.1775591  | 17.0 |
| 885866790.4100832  | 16.0 |
| 1128837891.6846883 | 15.0 |
| 1438449888.2876658 | 15.0 |
| 1832980710.8324375 | 13.0 |
| 2335721469.0901213 | 12.0 |
| 2976351441.6313133 | 10.0 |
| 3792690190.7322536 | 6.0  |
| 4832930238.571753  | 5.0  |
| 6158482110.6602545 | 3.0  |
| 7847599703.514623  | 1.0  |
|   10000000000.0    | 1.0  |
+--------------------+------+
[20 rows x 2 columns]



Out of this large range, we want to find the two ends of our desired narrow range of `l1_penalty`.  At one end, we will have `l1_penalty` values that have too few non-zeros, and at the other end, we will have an `l1_penalty` that has too many non-zeros.  

More formally, find:
* The largest `l1_penalty` that has more non-zeros than `max_nonzeros` (if we pick a penalty smaller than this value, we will definitely have too many non-zero weights)
    * Store this value in the variable `l1_penalty_min` (we will use it later)
* The smallest `l1_penalty` that has fewer non-zeros than `max_nonzeros` (if we pick a penalty larger than this value, we will definitely have too few non-zero weights)
    * Store this value in the variable `l1_penalty_max` (we will use it later)


*Hint: there are many ways to do this, e.g.:*
* Programmatically within the loop above
* Creating a list with the number of non-zeros for each value of `l1_penalty` and inspecting it to find the appropriate boundaries.

In [56]:
l1_penalty_min = handle_1['l1'][3]
l1_penalty_max = handle_1['l1'][18]


***QUIZ QUESTION.*** What values did you find for `l1_penalty_min` and `l1_penalty_max`, respectively? 

## Exploring the narrow range of values to find the solution with the right number of non-zeros that has lowest RSS on the validation set 

We will now explore the narrow region of `l1_penalty` values we found:

In [57]:
l1_penalty_values = np.linspace(l1_penalty_min,l1_penalty_max,20)

* For `l1_penalty` in `np.linspace(l1_penalty_min,l1_penalty_max,20)`:
    * Fit a regression model with a given `l1_penalty` on TRAIN data. Specify `l1_penalty=l1_penalty` and `l2_penalty=0.` in the parameter list. When you call `linear_regression.create()` make sure you set `validation_set = None`
    * Measure the RSS of the learned model on the VALIDATION set

Find the model that the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`.

In [62]:
penalty_set=np.logspace(1,7,num=13)
handle_2=turicreate.SFrame()
array1_1=np.array([])
array1_2=np.array([])
array1_3=np.array([])
print(penalty_set.size)
for l1_penalty in l1_penalty_values:
    model=turicreate.linear_regression.create(training,
                                              target='price',
                                              features=all_features,
                                             l1_penalty=l1_penalty,
                                             l2_penalty=0,
                                             validation_set=None,verbose = False)
    validation_output=model.predict(validation)
    error=validation_output-validation['price']
    rss=np.dot(error,error)
    
    array1_1=np.append(array1_1,[l1_penalty])
    array1_2=np.append(array1_2,[rss])
    array1_3=np.append(array1_3,[model.coefficients['value'].nnz()])
#     print(l1_penalty, '-->', rss)

13


In [64]:
handle_2['l1_penalty']=array1_1
handle_2['rss']=array1_2
handle_2['nnz']=array1_3
handle_2.sort('rss',ascending=True)
handle_2.print_rows(20)

+--------------------+--------------------+------+
|     l1_penalty     |        rss         | nnz  |
+--------------------+--------------------+------+
|  206913808.111479  | 630650082718674.5  | 18.0 |
|  609055171.027434  | 653769798188253.2  | 17.0 |
| 1011196533.9433889 | 689408481347851.5  | 16.0 |
| 1413337896.8593438 | 734291272144302.8  | 15.0 |
| 1815479259.7752988 | 793956426237549.9  | 13.0 |
| 2217620622.6912537 | 849982040203357.8  | 12.0 |
| 2619761985.6072083 | 913641680992875.9  | 11.0 |
| 3021903348.5231633 | 974448934124820.9  | 10.0 |
| 3424044711.4391184 | 1044917964832726.9 | 7.0  |
| 3826186074.3550735 | 1085334503093306.6 | 6.0  |
| 4228327437.2710285 | 1136859387334644.0 | 6.0  |
| 4630468800.186983  | 1206918203420764.5 | 6.0  |
| 5032610163.102938  | 1274425792050292.5 | 5.0  |
| 5434751526.018893  | 1334883722509688.8 | 4.0  |
| 5836892888.934848  | 1362827217871108.5 | 3.0  |
| 6239034251.850803  | 1393461129647987.0 | 3.0  |
| 6641175614.766758  | 14252546

***QUIZ QUESTIONS***
1. What value of `l1_penalty` in our narrow range has the lowest RSS on the VALIDATION set and has sparsity *equal* to `max_nonzeros`?
2. What features in this model have non-zero coefficients?

In [66]:
final_model=turicreate.linear_regression.create(training,
                                              target='price',
                                              features=all_features,
                                             l1_penalty=206913808.111479,
                                             l2_penalty=0,
                                             validation_set=None,verbose = False)
final_model.coefficients.print_rows(18)

+------------------+-------+----------------------+--------+
|       name       | index |        value         | stderr |
+------------------+-------+----------------------+--------+
|   (intercept)    |  None |  31564.606473324176  |  None  |
|     bedrooms     |  None |  7618.567764636495   |  None  |
| bedrooms_square  |  None |  750.1709548829191   |  None  |
|    bathrooms     |  None |  25026.177407617186  |  None  |
|   sqft_living    |  None |  38.935915253109776  |  None  |
| sqft_living_sqrt |  None |  1108.3863193708114  |  None  |
|     sqft_lot     |  None | -0.01774476275138417 |  None  |
|  sqft_lot_sqrt   |  None |  138.38532594587818  |  None  |
|      floors      |  None |  20124.746720019688  |  None  |
|  floors_square   |  None |  11975.497761696126  |  None  |
|    waterfront    |  None |  532082.0756272507   |  None  |
|       view       |  None |  88631.04255545625   |  None  |
|    condition     |  None |  6082.626008198674   |  None  |
|      grade       |  No